# Google Provider Guide 🌟

This notebook demonstrates how to use Google's Gemini models through different hosting options:
- **AI Studio**: Google's direct API service for developers
- **Vertex AI**: Enterprise-grade platform through Google Cloud

## Overview

Google offers Gemini models through multiple platforms, each designed for different use cases:

### AI Studio (Direct Google Hosting)
- **Easy setup** with simple API key authentication
- **Latest models** available immediately upon release
- **Developer-friendly** with minimal configuration
- **Free tier** available for experimentation
- **Global access** without regional restrictions

### Vertex AI (Google Cloud Hosting)
- **Enterprise features** including VPC integration
- **Regional deployment** for compliance and latency
- **Advanced monitoring** and logging capabilities
- **Custom model fine-tuning** options
- **IAM integration** for access control
- **Batch processing** and high-throughput scenarios

## Setup

First, let's load environment variables for authentication:

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## AI Studio (Direct Google Hosting)

### Authentication
AI Studio uses the `GEMINI_API_KEY` environment variable for authentication. You can get your API key from [Google AI Studio](https://aistudio.google.com/).

### Available Models
- `gemini-2.0-flash-001` - Latest and fastest Gemini model
- `gemini-1.5-pro-001` - Most capable model for complex tasks
- `gemini-1.5-flash-001` - Balanced performance and speed
- `gemini-1.0-pro` - Previous generation model

### Key Features
- **Multimodal capabilities** (text, images, audio, video)
- **Function calling** for tool integration
- **Large context windows** (up to 2M tokens)
- **Code generation** and execution
- **Reasoning capabilities** with chain-of-thought

### Resources
- [Python SDK Documentation](https://github.com/googleapis/python-genai)
- [AI Studio](https://aistudio.google.com/)
- [Model Documentation](https://ai.google.dev/models/gemini)

### Basic Message Example

In [2]:
from google import genai

# Initialize AI Studio client
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# Basic content generation
response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='Explain quantum computing in simple terms, using an analogy.'
)

print("=== AI Studio Response ===")
print(f"Model: {response.model_version}")

if response.candidates and len(response.candidates) > 0:
    candidate = response.candidates[0]
    print(f"Content: {candidate.content.parts[0].text}")
    print(f"Finish reason: {candidate.finish_reason}")
    
    # Safety ratings (if available)
    if candidate.safety_ratings:
        print(f"Safety ratings: {len(candidate.safety_ratings)} categories checked")

print(f"Usage: {response.usage_metadata.prompt_token_count} + {response.usage_metadata.candidates_token_count} = {response.usage_metadata.total_token_count} tokens")

# Full response structure
print("\n=== Full Response Structure ===")
response.model_dump()

=== AI Studio Response ===
Model: gemini-2.0-flash-001
Content: Okay, let's explain quantum computing with a coin analogy.

**Classical Computing (Regular Coins):**

Imagine you have a regular coin. It can be either Heads (0) or Tails (1). That's it.  A classical computer uses bits, which are like these coins. Each bit can only be in one state at a time – either 0 or 1.  Think of a light switch: it's either on (1) or off (0).  You can only do one thing at a time with each switch. If you want to represent the number 3, you need to use multiple switches turned on and off in a specific pattern (like 11 in binary).  Every calculation is done step-by-step, using these "on/off" switches in a determined sequence.

**Quantum Computing (Magical Spinning Coins):**

Now, imagine a magical coin that is spinning in the air.  While it's spinning, it's *both* Heads and Tails *at the same time*. It's not *either* heads *or* tails, but a **superposition** of both.  Think of it as a combination of both 

{'candidates': [{'content': {'parts': [{'video_metadata': None,
      'thought': None,
      'inline_data': None,
      'code_execution_result': None,
      'executable_code': None,
      'file_data': None,
      'function_call': None,
      'function_response': None,
      'text': 'Okay, let\'s explain quantum computing with a coin analogy.\n\n**Classical Computing (Regular Coins):**\n\nImagine you have a regular coin. It can be either Heads (0) or Tails (1). That\'s it.  A classical computer uses bits, which are like these coins. Each bit can only be in one state at a time – either 0 or 1.  Think of a light switch: it\'s either on (1) or off (0).  You can only do one thing at a time with each switch. If you want to represent the number 3, you need to use multiple switches turned on and off in a specific pattern (like 11 in binary).  Every calculation is done step-by-step, using these "on/off" switches in a determined sequence.\n\n**Quantum Computing (Magical Spinning Coins):**\n\nNow

### Advanced Configuration with Messages

For more complex interactions, you can use the message format with system instructions:

In [3]:
from google.genai import types

# Create a conversation with system instructions
system_instruction = "You are a helpful AI assistant specializing in creative writing. Always provide vivid, engaging responses."

messages = [
    types.Content(
        role="user",
        parts=[types.Part(text="Write a compelling opening paragraph for a mystery novel set in a futuristic city.")]
    )
]

# Configuration for more control
config = types.GenerateContentConfig(
    system_instruction=system_instruction,
    temperature=0.8,  # More creative responses
    top_p=0.95,
    max_output_tokens=200,
    candidate_count=1
)

advanced_response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=messages,
    config=config
)

print("=== Advanced Configuration Response ===")
print(f"Model: {advanced_response.model_version}")
if advanced_response.candidates:
    print(f"Content: {advanced_response.candidates[0].content.parts[0].text}")
    print(f"Average log probability: {advanced_response.candidates[0].avg_logprobs:.4f}")

=== Advanced Configuration Response ===
Model: gemini-2.0-flash-001
Content: The neon rain tasted like static and regret on my tongue as I stood on the Skywalk, the shimmering arteries of Neo-Kyoto pulsing beneath my worn boots. A glitch in the matrix of light, a flicker in the holographic geishas advertising synth-sake, that was all it took to distract me for a heartbeat. A heartbeat was all it took for her to fall. Now, all that remained of Anya Volkov, the darling of the Zaibatsu elite, was a crimson stain blooming on the chrome pavement a thousand stories below, and the nagging suspicion that gravity hadn't acted alone.

Average log probability: -0.5384


## Function Calling (Tool Use)

Google's Gemini models support function calling for interacting with external tools and APIs.

### Key Features:
- **Function declarations** using JSON schemas
- **Parallel function calls** for efficiency
- **Automatic function calling** mode
- **Complex parameter schemas** with nested objects

### Force function calling
The Gemini API lets you control how the model uses the provided tools (function declarations). Specifically, you can set the mode within the function_calling_config.

- AUTO (Default): The model decides whether to generate a natural language response or suggest a function call based on the prompt and context. This is the most flexible mode and recommended for most scenarios.
- ANY: The model is constrained to always predict a function call and guarantee function schema adherence. If allowed_function_names is not specified, the model can choose from any of the provided function declarations. If allowed_function_names is provided as a list, the model can only choose from the functions in that list. Use this mode when you require a function call in response to every prompt (if applicable).
- NONE: The model is prohibited from making function calls. This is equivalent to sending a request without any function declarations. Use this to temporarily disable function calling without removing your tool definitions.

### Single Function Call Example

In [4]:
# Define a weather function
weather_function = {
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Tokyo, Japan"
            },
            "units": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Temperature units"
            }
        },
        "required": ["location"]
    }
}

# Create function tool
tools = types.Tool(function_declarations=[weather_function])

# Create the config
tool_config = types.ToolConfig(
    function_calling_config=types.FunctionCallingConfig(
        mode="ANY", allowed_function_names=["get_weather"]
    )
)

config = types.GenerateContentConfig(tools=[tools], tool_config=types.ToolConfig)

# Initial request with function
messages = [
    types.Content(
        role="user",
        parts=[types.Part(text="What is the weather like in Tokyo today?")]
    )
]

function_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=messages,
    config=config
)

print("=== Function Call Response ===")
print(f"Model: {function_response.model_version}")

if function_response.candidates and len(function_response.candidates) > 0:
    candidate = function_response.candidates[0]
    
    for i, part in enumerate(candidate.content.parts):
        print(f"\nPart {i+1}: {part.__class__.__name__}")
        
        if part.text:
            print(f"  Text: {part.text}")
        elif part.function_call:
            print(f"  Function: {part.function_call.name}")
            print(f"  Arguments: {dict(part.function_call.args)}")

# Add assistant's response to conversation
messages.append(candidate.content)

print(f"\nMessages so far: {len(messages)}")

=== Function Call Response ===
Model: gemini-2.0-flash

Part 1: Part
  Function: get_weather
  Arguments: {'location': 'Tokyo, Japan'}

Messages so far: 2


### Function Execution and Response

In [5]:
# Simulate function execution
def execute_weather_function(location, units="celsius"):
    """Simulate getting weather data"""
    if "tokyo" in location.lower():
        if units == "celsius":
            return "22°C, partly cloudy with light breeze"
        else:
            return "72°F, partly cloudy with light breeze"
    else:
        return f"Weather data not available for {location}"

# Find function call in the response
function_call = None
if function_response.candidates:
    for part in function_response.candidates[0].content.parts:
        if part.function_call:
            function_call = part.function_call
            break

if function_call:
    # Execute the function
    location = function_call.args.get("location")
    units = function_call.args.get("units", "celsius")
    weather_result = execute_weather_function(location, units)
    
    print(f"=== Function Execution ===")
    print(f"Function: {function_call.name}")
    print(f"Location: {location}")
    print(f"Units: {units}")
    print(f"Result: {weather_result}")
    
    # Create function response
    function_response_part = types.Part.from_function_response(
        name=function_call.name,
        response={"result": weather_result}
    )
    
    messages.append(
        types.Content(role="user", parts=[function_response_part])
    )
    
    # Get final response with function results
    final_response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=messages,
        config=config
    )
    
    print(f"\n=== Final Response ===")
    if final_response.candidates:
        print(f"Gemini's response: {final_response.candidates[0].content.parts[0].text}")
else:
    print("No function calls found in response")

=== Function Execution ===
Function: get_weather
Location: Tokyo, Japan
Units: celsius
Result: 22°C, partly cloudy with light breeze

=== Final Response ===
Gemini's response: It is 22°C in Tokyo, partly cloudy with a light breeze.



### Multiple Functions Example

In [6]:
# Define multiple functions
calculator_function = {
    "name": "calculator",
    "description": "Perform basic mathematical operations",
    "parameters": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "enum": ["add", "subtract", "multiply", "divide"],
                "description": "The mathematical operation"
            },
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"}
        },
        "required": ["operation", "a", "b"]
    }
}

time_function = {
    "name": "get_current_time",
    "description": "Get the current time in a specified timezone",
    "parameters": {
        "type": "object",
        "properties": {
            "timezone": {
                "type": "string",
                "description": "Timezone (e.g., UTC, America/New_York, Asia/Tokyo)",
                "default": "UTC"
            }
        },
        "required": []
    }
}

# Create tools with multiple functions
multi_tools = types.Tool(function_declarations=[
    weather_function,
    calculator_function,
    time_function
])
multi_config = types.GenerateContentConfig(tools=[multi_tools])

# Request that might use multiple functions
multi_function_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[
        types.Content(
            role="user",
            parts=[types.Part(text="Calculate 35 * 12, and also tell me what time it is in Tokyo")]
        )
    ],
    config=multi_config
)

print("=== Multiple Functions Response ===")
print(f"Model: {multi_function_response.model_version}")

if multi_function_response.candidates:
    candidate = multi_function_response.candidates[0]
    
    print(f"\nContent parts: {len(candidate.content.parts)}")
    function_calls = []
    
    for i, part in enumerate(candidate.content.parts):
        print(f"\nPart {i+1}: {part.__class__.__name__}")
        
        if part.text:
            print(f"  Text: {part.text}")
        elif part.function_call:
            print(f"  Function: {part.function_call.name}")
            print(f"  Arguments: {dict(part.function_call.args)}")
            function_calls.append(part.function_call)
    
    print(f"\nTotal function calls: {len(function_calls)}")
    print(f"Usage: {multi_function_response.usage_metadata.prompt_token_count} + {multi_function_response.usage_metadata.candidates_token_count} = {multi_function_response.usage_metadata.total_token_count} tokens")

=== Multiple Functions Response ===
Model: gemini-2.0-flash

Content parts: 2

Part 1: Part
  Function: calculator
  Arguments: {'a': 35, 'b': 12, 'operation': 'multiply'}

Part 2: Part
  Function: get_current_time
  Arguments: {'timezone': 'Asia/Tokyo'}

Total function calls: 2
Usage: 113 + 16 = 129 tokens


## Images and PDF-files

### Images
First let's try sending an image from local storage

In [7]:
import requests

image1_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"

# Download image from URL instead of trying to open it as a local file
response = requests.get(image1_url)
image_bytes = response.content

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=[
        types.Part.from_bytes(
            data=image_bytes,
            mime_type='image/jpeg',
        ),
        "What's this image about? "
    ]
)

print(response.text)

The image features a close-up of a black ant on a rough, beige surface. The ant is standing with its front legs raised, as if in a defensive or inquisitive posture. The background is blurred, with hints of brown and red, focusing attention on the ant itself. The lighting and macro focus highlight the details of the ant's body, including its segmented abdomen and spiny legs.



You can also just pass a URL if it's a public image

In [8]:
image_path = "https://goo.gle/instrument-img"
image_bytes = requests.get(image_path).content
image = types.Part.from_bytes(
  data=image_bytes, mime_type="image/jpeg"
)

response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=["What is this image?", image],
)

print(response.text)

The image shows the console of a pipe organ. It is made of wood and has multiple keyboards, stop controls, and a pedalboard.


### PDF-Files

In [9]:
prompt = "Summarize this document"
response = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=[
      types.Part.from_bytes(
        data=open("./assets/gameboy_color.pdf", "rb").read(),
        mime_type='application/pdf',
      ),
      prompt])
print(response.text)

This document is an instruction booklet for the Nintendo Game Boy Color video game system. It includes:

*   **Introduction:** Features and capabilities of the Game Boy Color.
*   **Components:** Lists and describes the various parts of the system.
*   **Installation:** Instructions on how to install batteries.
*   **Cautions:** Information on which accessories are not compatible.
*   **Game Paks:** Describes the different types of Game Paks that can be used with the Game Boy Color.
*   **Usage:** Instructions on how to insert and remove Game Paks, and how to turn the system on/off.
*   **Screen Color:** Instructions on how to change the screen color for original Game Boy Game Paks.
*   **Multiplayer:** Instructions on how to play two-player games.
*   **Troubleshooting:** Solutions to common problems.
*   **Warranty & Service:** Information on the warranty and how to get service.
*   **Parts List:** A list of parts that can be ordered.


In [10]:
myfile = client.files.upload(file="./assets/gameboy_color.pdf")

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=["Describe this PDF in 2 sentences", myfile]
)

print(response.text)

This is the instruction booklet for the original Game Boy Color video game system by Nintendo that discusses the components, batteries, game paks, multi-player options, troubleshooting, warranty, and more. It also includes a warning about not using the Game Boy Rechargeable Battery Pack.


#### Word Documents (.docx)

In [6]:
import mammoth
from google.genai import types

# Convert with default options
with open("./assets/order.docx", "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    html = result.value  # The HTML
    messages = result.messages  # Any warnings

prompt = "Summarize this document"
response = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=[
      types.Part.from_text(
        text=html,
      ),
      prompt])
print(response.text)



Here's a summary of the provided document:

**Document Type:** Purchase Order

**Issuing Company:** &lt;Company Name&gt; (&lt;Address&gt;, &lt;Contact Number&gt;, &lt;Default Email Address&gt;, &lt;Website URL&gt;)

**Purchase Order Number:** 23781
**Date:** nn/dd/yyyy

**Vendor:**
*   **Name:** &lt;Sales Person&gt;
*   **Company Name:** &lt;Company Name&gt;
*   **Address:** &lt;Address&gt;
*   **Phone:** &lt;Phone&gt;
*   **Email:** &lt;Email Address&gt;

**Customer:**
*   **Name:** John Smith
*   **Company Name:** Redline Auto Center
*   **Address:** 16040 S. US 27, Lansing, Michigan 48906
*   **Phone:** 517-367-7010
*   **Email:** johnsmith@redline.com

**Shipping Terms:** Freight on Board
**Shipping Method:** Air & Land

**Items Ordered:**

*   **304-98632:** Brake Discs, Pads & Calipers (Quantity: 4, Unit Price: 111.36, Amount: 445.44)
*   **501-35587:** Control Arm (Quantity: 2, Unit Price: 60.93, Amount: 121.86)
*   **886-19386:** Suspension Lift Kit (Quantity: 2, Unit Price: 39

## Vertex AI Hosting

### Setup and Authentication
Vertex AI requires Google Cloud project configuration and uses service account authentication or Application Default Credentials (ADC).

### Key Differences:
- **Project ID**: Must specify your Google Cloud project
- **Location**: Choose deployment region for compliance/latency
- **Enterprise Features**: VPC integration, audit logging, and advanced monitoring
- **IAM Integration**: Fine-grained access control
- **Custom Models**: Support for fine-tuned and custom models

### Available Regions:
- `us-central1` - United States (primary)
- `us-east4` - United States (secondary)
- `europe-west1` - Europe (primary)
- `asia-northeast1` - Asia Pacific (Tokyo)
- See [Vertex AI Locations](https://cloud.google.com/vertex-ai/docs/general/locations) for complete list

### Resources:
- [Vertex AI Documentation](https://cloud.google.com/vertex-ai/docs)
- [Gemini on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini)
- [Google Cloud Console](https://console.cloud.google.com/vertex-ai)

### Basic Vertex AI Example

In [11]:
# Initialize Vertex AI client
project_id = os.environ.get("GCP_PROJECT_ID")
location = os.environ.get("GCP_LOCATION", "us-central1")

print(f"Vertex AI Configuration:")
print(f"  Project: {project_id}")
print(f"  Location: {location}")

vertex_client = genai.Client(
    vertexai=True,
    project=project_id,
    location=location
)

# Basic content generation on Vertex AI
vertex_response = vertex_client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents='Explain the benefits of using Vertex AI over AI Studio for enterprise applications.'
)

print("\n=== Vertex AI Response ===")
print(f"Model: {vertex_response.model_version}")

if vertex_response.candidates and len(vertex_response.candidates) > 0:
    candidate = vertex_response.candidates[0]
    print(f"Content: {candidate.content.parts[0].text}")
    print(f"Finish reason: {candidate.finish_reason}")

print(f"Usage: {vertex_response.usage_metadata.prompt_token_count} + {vertex_response.usage_metadata.candidates_token_count} = {vertex_response.usage_metadata.total_token_count} tokens")

# Additional Vertex AI metadata
if hasattr(vertex_response, 'create_time'):
    print(f"Create time: {vertex_response.create_time}")
if hasattr(vertex_response, 'response_id'):
    print(f"Response ID: {vertex_response.response_id}")

print("\n=== Full Response Structure ===")
vertex_response

Vertex AI Configuration:
  Project: vectrix-401014
  Location: europe-west1

=== Vertex AI Response ===
Model: gemini-2.0-flash-001
Content: While both Vertex AI and AI Studio are Google Cloud Platform (GCP) offerings for machine learning (ML) development, Vertex AI offers significant advantages over AI Studio for enterprise applications. Here's a breakdown of the benefits:

**Vertex AI: The Enterprise-Grade ML Platform**

Vertex AI is a unified platform designed for the entire ML lifecycle, from data preparation to model deployment and monitoring. It provides a comprehensive set of tools and services for building, training, and deploying ML models at scale.

**Key Benefits of Vertex AI over AI Studio for Enterprise Applications:**

*   **Full ML Lifecycle Management:**
    *   **Vertex AI:** Handles the entire ML lifecycle from data ingestion, preparation, and labeling to model training, evaluation, deployment, monitoring, and management.
    *   **AI Studio:** Primarily focused on mo

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text="While both Vertex AI and AI Studio are Google Cloud Platform (GCP) offerings for machine learning (ML) development, Vertex AI offers significant advantages over AI Studio for enterprise applications. Here's a breakdown of the benefits:\n\n**Vertex AI: The Enterprise-Grade ML Platform**\n\nVertex AI is a unified platform designed for the entire ML lifecycle, from data preparation to model deployment and monitoring. It provides a comprehensive set of tools and services for building, training, and deploying ML models at scale.\n\n**Key Benefits of Vertex AI over AI Studio for Enterprise Applications:**\n\n*   **Full ML Lifecycle Management:**\n    *   **Vertex AI:** Handles the entire ML lifecycle from data ingestion, preparation, and labeling to mod

### Vertex AI Function Calling

Function calling works identically on Vertex AI as with AI Studio:

In [12]:
# Function calling works the same on Vertex AI
vertex_tools = types.Tool(function_declarations=[calculator_function])
vertex_config = types.GenerateContentConfig(tools=[vertex_tools])

vertex_function_response = vertex_client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents=[
        types.Content(
            role="user",
            parts=[types.Part(text="Calculate 88 divided by 11")]
        )
    ],
    config=vertex_config
)

print("=== Vertex AI Function Call ===")
print(f"Model: {vertex_function_response.model_version}")

if vertex_function_response.candidates:
    candidate = vertex_function_response.candidates[0]
    
    for part in candidate.content.parts:
        if part.text:
            print(f"Text: {part.text}")
        elif part.function_call:
            print(f"Function: {part.function_call.name}")
            args = dict(part.function_call.args)
            print(f"Operation: {args['a']} {args['operation']} {args['b']}")
            print(f"Arguments: {args}")

print(f"\n✅ Function calling works identically on Vertex AI!")

=== Vertex AI Function Call ===
Model: gemini-2.0-flash-001
Function: calculator
Operation: 88 divide 11
Arguments: {'a': 88, 'operation': 'divide', 'b': 11}

✅ Function calling works identically on Vertex AI!


## Comparison: AI Studio vs Vertex AI

Let's compare the same request across both hosting options:

In [13]:
test_content = "Explain artificial intelligence in exactly 2 sentences."

print("=== Comparison: AI Studio vs Vertex AI ===")

# AI Studio
studio_response = client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=test_content
)

print("\n🟡 AI Studio:")
print(f"Model: {studio_response.model_version}")
if studio_response.candidates:
    print(f"Response: {studio_response.candidates[0].content.parts[0].text}")
print(f"Tokens: {studio_response.usage_metadata.prompt_token_count} + {studio_response.usage_metadata.candidates_token_count}")

# Vertex AI
vertex_response = vertex_client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=test_content
)

print("\n🔵 Vertex AI:")
print(f"Model: {vertex_response.model_version}")
if vertex_response.candidates:
    print(f"Response: {vertex_response.candidates[0].content.parts[0].text}")
print(f"Tokens: {vertex_response.usage_metadata.prompt_token_count} + {vertex_response.usage_metadata.candidates_token_count}")

print("\n💡 Both use identical APIs and response formats!")
print("💡 Key differences: authentication, enterprise features, and regional deployment")

=== Comparison: AI Studio vs Vertex AI ===

🟡 AI Studio:
Model: gemini-2.0-flash-001
Response: Artificial intelligence is the ability of a computer or machine to mimic human cognitive functions, such as learning, problem-solving, and decision-making. It involves developing algorithms and models that enable machines to perform tasks that typically require human intelligence.

Tokens: 9 + 48

🔵 Vertex AI:
Model: gemini-2.0-flash-001
Response: Artificial intelligence is the simulation of human intelligence processes by computer systems. These processes include learning, reasoning, and problem-solving, allowing machines to perform tasks that typically require human intelligence.

Tokens: 9 + 37

💡 Both use identical APIs and response formats!
💡 Key differences: authentication, enterprise features, and regional deployment
